# 1. Configuración inicial y librerías necesarias

### 1. Instalación y configuración de TensorFlow/Keras

Asegúrate de que TensorFlow y Keras estén instalados:





In [ ]:
#!pip install tensorflow

Verifica la instalación importando TensorFlow en un entorno Python:

In [ ]:
import tensorflow as tf
from keras import backend as K

print(tf.__version__)
# Reiniciar el backend para liberar memoria
tf.keras.backend.clear_session()
K.clear_session()

2.17.1


### 2. Manejo de datasets con MNIST (u otro dataset similar)

Carga el dataset MNIST, que contiene imágenes de dígitos escritos a mano:

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, _), (_, _) = mnist.load_data()
x_train = (x_train.astype('float32') - 127.5) / 127.5  # Normalizar entre -1 y 1
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)  # Redimensionar
print(f"Dataset shape: {x_train.shape}")

Dataset shape: (60000, 28, 28, 1)


Divide los datos en lotes pequeños:

In [ ]:
batch_size = 16 # Mejor 64
dataset = tf.data.Dataset.from_tensor_slices(
    x_train).shuffle(60000).batch(batch_size,drop_remainder=True)

# 2. Implementación básica de una GAN

### 1. Creación del generador

Diseña una red neuronal que transforma un vector de ruido en una imagen

In [ ]:
from tensorflow.keras import layers

def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, input_dim=100, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(28 * 28 * 1, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])
    return model


### 2. Creación del discriminador

Diseña una red que clasifica imágenes como reales o generadas

In [ ]:
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model


### 3. Combinar el generador y el discriminador para formar la GAN

Conecta ambas redes en un modelo compuesto:

In [ ]:
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy',
                      metrics=['accuracy'])
discriminator.trainable = False
gan_input = layers.Input(shape=(100,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# 3. Entrenamiento de la GAN

### 1. Inicialización del bucle de entrenamiento

Definir las funciones de generación de ruido y lotes de datos

In [ ]:
import matplotlib.pyplot as plt

def generate_noise(batch_size, noise_dim):
    return np.random.normal(0, 1, (batch_size, noise_dim))

def plot_images(generator, noise_dim, examples=16):
    noise = generate_noise(examples, noise_dim)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Desnormalizar
    fig, axes = plt.subplots(4, 4, figsize=(4, 4))
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(generated_images[i, :, :, 0], cmap='gray')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

### 2. Bucle de entrenamiento

Alterna entre entrenar el discriminador y el generador:

In [ ]:
import gc
#epochs = 10000
epochs = 4
noise_dim = 100

for epoch in range(epochs):
    for real_images in dataset:
        # Entrenar discriminador
        noise = generate_noise(batch_size, noise_dim)
        fake_images = generator.predict(noise)
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Entrenar generador
        noise = generate_noise(batch_size, noise_dim)
        misleading_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, misleading_labels)

    # Mostrar progreso
    g_loss_value = g_loss[0] if isinstance(g_loss, list) else g_loss
    if epoch % 2 == 0:
        print(f"Epoch {epoch} | D Loss: {d_loss[0]:.4f} | G Loss: {g_loss_value:.4f}")
        plot_images(generator, noise_dim)

    # Limpiar memoria después de cada época
    gc.collect()
    tf.keras.backend.clear_session()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

# 4. Generación de imágenes sintéticas

Al final del entrenamiento, genera imágenes a partir del generador:

In [ ]:
plot_images(generator, noise_dim)

Observa los resultados:
*   ¿Las imágenes son realistas?
*   ¿Qué mejoras podrían implementarse para mejorar la calidad?



# 5. Actividades: Mejora del Modelo
### 1. Aumentar la complejidad de la red

- Agregar más capas al generador o discriminador.
- Cambiar las funciones de activación (por ejemplo, usar LeakyReLU en lugar de ReLU).

### 2. Ajustar hiperparámetros

- Experimentar con diferentes tasas de aprendizaje.
- Cambiar el tamaño del vector de ruido.